In [1]:
import json
import numpy as np
from matplotlib import pyplot as plt
from plotly import graph_objs as go

In [2]:
with open('300wLP_anno_tr.json') as f:
    data = json.load(f)

In [3]:
def plot_3d(landmarks):
    
    landmarks_3d = np.array(landmarks).transpose()
    
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=landmarks_3d[0],
        y=landmarks_3d[1],
        z=landmarks_3d[2],
        mode='markers',
        marker=dict(size=3, color='red'),
        name='3D Landmarks'
    ))

    # Update layout for interactive controls
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
            aspectmode='cube',  # Cube aspect ratio for equal scaling
            camera=dict(up=dict(x=0, y=0, z=1), eye=dict(x=0, y=0, z=-2)),  # Initial camera position
        ),
        title='Interactive 3D Facial Landmarks Visualization'
    )

    # Show the interactive plot in Jupyter Notebook or in a separate browser window
    fig.show()

    #plt.scatter(landmarks_2d[0], landmarks_2d[1], c='red', s=3)
    #plt.imshow(image)
    plt.show()

In [4]:
index = 0
landmarks = data[index]['landmarks']
plot_3d(landmarks)